##use sympy package to interpolate shorelines of missing years

In [1]:
import sympy as sy
import sympy.geometry as gm

In [26]:
import geopy

In [6]:
import rasterio


ModuleNotFoundError: No module named 'gdal'

In [7]:
from osgeo import gdal

In [7]:
import geopandas as gpd

In [50]:
import math

In [5]:
coast_1860=gpd.read_file("1860_coastline.geojson")
coast_1860

,Shape_Leng,geometry
0,0.181588,"MULTILINESTRING ((-124.11763 46.87711, -124.11..."


In [38]:
coast_1911=gpd.read_file("1911_coastline.geojson")
coast_1911

,FEATURE_ID,FEATURE,SOURCE_ID,DATA_SOURC,SRC_DATE,HOR_ACC,INFORM,RESOLUTION,ATTRIBUTE,CLASS,F_CODE,EX_METH,EXTRACT_TE,geometry
0,946684.0,15.0,T03044,P,19090101,0,None,0.0,Natural.Apparent.Marsh Or Swamp,SHORELINE,None,M,P,"MULTILINESTRING ((-124.02782 46.90018, -124.02..."
1,946685.0,26.0,T03044,P,19090101,0,None,0.0,Undetermined.Approximate,SHORELINE,None,M,P,"MULTILINESTRING ((-124.02703 46.90027, -124.02..."
2,946686.0,15.0,T03044,P,19090101,0,None,0.0,Natural.Apparent.Marsh Or Swamp,SHORELINE,None,M,P,"MULTILINESTRING ((-124.02668 46.90046, -124.02..."
3,946687.0,205.0,T03044,P,19090101,0,None,0.0,User Added Line,CARTOGRAPHIC LIMIT,None,M,P,"MULTILINESTRING ((-124.16458 47.02184, -124.16..."
4,946266.0,15.0,T03044,P,19090101,0,None,0.0,Natural.Apparent.Marsh Or Swamp,SHORELINE,None,M,P,"MULTILINESTRING ((-124.00882 46.91098, -124.00..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
806,947660.0,15.0,T03155,P,19110101,0,None,0.0,Natural.Apparent.Marsh Or Swamp,SHORELINE,None,M,P,"MULTILINESTRING ((-124.04030 46.86526, -124.04..."
807,947661.0,15.0,T03155,P,19110101,0,None,0.0,Natural.Apparent.Marsh Or Swamp,SHORELINE,None,M,P,"MULTILINESTRING ((-124.07154 46.85415, -124.07..."
808,947662.0,26.0,T03155,P,19110101,0,None,0.0,Undetermined.Approximate,SHORELINE,None,M,P,"MULTILINESTRING ((-124.07524 46.84725, -124.07..."
809,947202.0,26.0,T03155,P,19110101,0,None,0.0,Undetermined.Approximate,SHORELINE,None,M,P,"MULTILINESTRING ((-124.06673 46.84068, -124.06..."


In [15]:
#year 1911 has two geojson files, combine them.
f_coast_1911=gpd.read_file("f_1911_coastline.geojson")
f_coast_1911bay=gpd.read_file("f_1911_coastline_bay.geojson")
combined_1911=f_coast_1911.append(f_coast_1911bay)
combined_1911

,FEATURE_ID,FEATURE,SOURCE_ID,DATA_SOURC,SRC_DATE,HOR_ACC,INFORM,RESOLUTION,ATTRIBUTE,CLASS,F_CODE,EX_METH,EXTRACT_TE,geometry
0,946684.0,15.0,T03044,P,19090101,0,None,0.0,Natural.Apparent.Marsh Or Swamp,SHORELINE,None,M,P,"MULTILINESTRING ((-124.02782 46.90018, -124.02..."
1,946685.0,26.0,T03044,P,19090101,0,None,0.0,Undetermined.Approximate,SHORELINE,None,M,P,"MULTILINESTRING ((-124.02703 46.90027, -124.02..."
2,946686.0,15.0,T03044,P,19090101,0,None,0.0,Natural.Apparent.Marsh Or Swamp,SHORELINE,None,M,P,"MULTILINESTRING ((-124.02668 46.90046, -124.02..."
3,946266.0,15.0,T03044,P,19090101,0,None,0.0,Natural.Apparent.Marsh Or Swamp,SHORELINE,None,M,P,"MULTILINESTRING ((-124.00882 46.91098, -124.00..."
4,946688.0,20.0,T03044,P,19090101,0,None,0.0,Natural.Mean High Water,SHORELINE,None,M,P,"MULTILINESTRING ((-124.16458 47.02184, -124.16..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
641,947660.0,15.0,T03155,P,19110101,0,None,0.0,Natural.Apparent.Marsh Or Swamp,SHORELINE,None,M,P,"MULTILINESTRING ((-124.04030 46.86526, -124.04..."
642,947661.0,15.0,T03155,P,19110101,0,None,0.0,Natural.Apparent.Marsh Or Swamp,SHORELINE,None,M,P,"MULTILINESTRING ((-124.07154 46.85415, -124.07..."
643,947662.0,26.0,T03155,P,19110101,0,None,0.0,Undetermined.Approximate,SHORELINE,None,M,P,"MULTILINESTRING ((-124.07524 46.84725, -124.07..."
644,947202.0,26.0,T03155,P,19110101,0,None,0.0,Undetermined.Approximate,SHORELINE,None,M,P,"MULTILINESTRING ((-124.06673 46.84068, -124.06..."


In [25]:
# read all coastline geojson files into a dictionary
year_with_data=[1860,1911,1922,1926,1941,1953,1954,2006,2007,2016]
dict_coast={}
for year in year_with_data:
    if year==1860:
        key_entry="coast_1860"
        dict_coast[key_entry]=gpd.read_file("1860_coastline.geojson")
    elif year in [1926,1941,1954,2006,2016]:
        key_entry="coast_"+str(year)
        dict_coast[key_entry]=gpd.read_file("f_"+str(year)+"_coastline.geojson")
    elif year in [1922,1953,2007]:
        key_entry="coast_"+str(year)
        dict_coast[key_entry]=gpd.read_file("f_"+str(year)+"_coastline_bay.geojson")
    else:
        key_entry="coast_"+str(year)
        dict_coast[key_entry]=combined_1911

In [26]:
#can be deleted later
len(dict_coast)

10

In [27]:
#can be deleted later
dict_coast["coast_1911"]

,FEATURE_ID,FEATURE,SOURCE_ID,DATA_SOURC,SRC_DATE,HOR_ACC,INFORM,RESOLUTION,ATTRIBUTE,CLASS,F_CODE,EX_METH,EXTRACT_TE,geometry
0,946684.0,15.0,T03044,P,19090101,0,None,0.0,Natural.Apparent.Marsh Or Swamp,SHORELINE,None,M,P,"MULTILINESTRING ((-124.02782 46.90018, -124.02..."
1,946685.0,26.0,T03044,P,19090101,0,None,0.0,Undetermined.Approximate,SHORELINE,None,M,P,"MULTILINESTRING ((-124.02703 46.90027, -124.02..."
2,946686.0,15.0,T03044,P,19090101,0,None,0.0,Natural.Apparent.Marsh Or Swamp,SHORELINE,None,M,P,"MULTILINESTRING ((-124.02668 46.90046, -124.02..."
3,946266.0,15.0,T03044,P,19090101,0,None,0.0,Natural.Apparent.Marsh Or Swamp,SHORELINE,None,M,P,"MULTILINESTRING ((-124.00882 46.91098, -124.00..."
4,946688.0,20.0,T03044,P,19090101,0,None,0.0,Natural.Mean High Water,SHORELINE,None,M,P,"MULTILINESTRING ((-124.16458 47.02184, -124.16..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,949585.0,26.0,T03224,P,19110101,0,None,0.0,Undetermined.Approximate,SHORELINE,None,M,P,"MULTILINESTRING ((-124.04670 46.71801, -124.04..."
58,949587.0,26.0,T03224,P,19110101,0,None,0.0,Undetermined.Approximate,SHORELINE,None,M,P,"MULTILINESTRING ((-124.03810 46.71697, -124.03..."
59,949588.0,26.0,T03224,P,19110101,0,None,0.0,Undetermined.Approximate,SHORELINE,None,M,P,"MULTILINESTRING ((-124.04088 46.72065, -124.04..."
60,950107.0,26.0,T03224,P,19110101,0,None,0.0,Undetermined.Approximate,SHORELINE,None,M,P,"MULTILINESTRING ((-123.97364 46.66091, -123.97..."


In [36]:
# a function to get the points from the multilinestring. store them in a dictionary, and the value in the dictionary has two dimensions, the first dimension is the line, and the second dimension is the point

dict_points={}

def get_points(year_of_multistrings):
    lines=dict_coast[year_of_multistrings]["geometry"]
    dict_points[year_of_multistrings]=[]
    for multiline in lines:
        for line in multiline:
            line_point_collection=[]
            for point in line.coords:
                line_point_collection.append(point)
            dict_points[year_of_multistrings].append(line_point_collection)
    


In [37]:
#write a loop to get all the points from the multilinestring of each year
for y in year_with_data:
    get_points("coast_"+str(y))

/var/folders/33/68l3prl56lsc37gr_prx3v_00000gn/T/ipykernel_25610/526783759.py:9: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for line in multiline:


In [43]:
#can be deleted later
dict_points.keys()
#dict_points["coast_1911"]

dict_keys(['coast_1860', 'coast_1911', 'coast_1922', 'coast_1926', 'coast_1941', 'coast_1953', 'coast_1954', 'coast_2006', 'coast_2007', 'coast_2016'])

In [34]:
#get the level of the nested list
def nest_level(obj):

    # Not a list? So the nest level will always be 0:
    if type(obj) != list:
        return 0

    # Now we're dealing only with list objects:

    max_level = 0
    for item in obj:
        # Getting recursively the level for each item in the list,
        # then updating the max found level:
        max_level = max(max_level, nest_level(item))

    # Adding 1, because 'obj' is a list (here is the recursion magic):
    return max_level + 1

In [46]:
nest_level([1,2,3,4,5,6,7,8,9,10,[1,4,5],[1,[3]]])

3

In [45]:
for k in dict_points.keys():
    print(k)
    print(nest_level(dict_points[k]))


coast_1860
2
coast_1911
2
coast_1922
2
coast_1926
2
coast_1941
2
coast_1953
2
coast_1954
2
coast_2006
2
coast_2007
2
coast_2016
2


## All coastline data has two levels. The lowest level is point given as tuple

In [47]:
# for example, get a point in the coast_2006
dict_points["coast_2006"][0][0]

(-123.95717, 46.9216231)

In [2]:
#get the point of intersection when drawing the vertical line from one coastline to the other.

from sympy import *


In [ ]:

#the function to get the intersection point of vertical line from one line to the other, start2 is the starting point of the vertical line.
def intersect_pt(start1_lon,start1_lat,start2_lon,start2_lat,target1_lon,target1_lat,target2_lon,target2_lat):

    #when the slope of the original line is not vertical nor horizontal
    if start1_lon != start2_lon and start1_lat != start2_lat:
        k_start=(start2_lat-start1_lat)/(start2_lon-start1_lon)  #slope of the original line k_start
        #when the slope of the target line is not vertical nor horizontal
        if target1_lon != target2_lon and target1_lat != target2_lat:
            k_target=(target2_lat-target1_lat)/(target2_lon-target1_lon)
            x, y= symbols("x y")
            eq = [(y-target1_lat)/(x-target1_lon)-k_target,(y-start2_lat)/(x-start2_lon)+1/k_start]
            result = solve(eq, [x, y])
            return_result=(result[x],result[y])
        #when the slope of the target line is horizontal
        elif target1_lat==target2_lat:
            y=target1_lat
            x=symbols("x")
            eq=[(y-start2_lat)/(x-start2_lon)+1/k_start]
            result = solve(eq, [x])
            return_result=(result[x],y)
        #when the slope of the target line is vertical
        else:
            x=target1_lon
            y=symbols("y")
            eq=[(y-start2_lat)/(x-start2_lon)+1/k_start]
            result = solve(eq, [y])
            return_result=(x,result[y])


    #when the slope of the original line is infinite:
    elif start1_lon==start2_lon:
        #when the slope of the original is infinite and the slope of the target line is not 0.
        if target1_lat!=target2_lat:
            y=start2_lat
            x=symbols("x")
            eq=[(target2_lat-y)/(target2_lat-target1_lat)-(target2_lon-x)/(target2_lon-target1_lon)]
            result = solve(eq, [x])
            return_result=(result[x],y)
        #when the slope of the original is infinite and the slope of the target is 0, there should not be an intersection point
        else:
            return_result = "No intersection"

    #when the slope of the original line is 0:
    else:
        #when the slope of the original is 0 and the slope of the target line is not infinite.
        if target1_lon!=target2_lon:
            x=start2_lon
            y=symbols("y")
            eq=[(target2_lat-y)/(target2_lat-target1_lat)-(target2_lon-x)/(target2_lon-target1_lon)]
            result = solve(eq, [y])
            return_result=(x,result[y])
        #when the slope of the original is 0 and the slope of the target is infinite, there should not be an intersection point
        else:
            return_result = "No intersection"

    return return_result


In [ ]:
#the function to get the section where the vertical line intersects the target line.
def plot_coastline(year):
    # get which year interval the year is in
    if year>=2016:
        return "No data"

    for i in range(len(year_with_data)):
        if year_with_data[i] <= year < year_with_data[i+1]:
            year_interval_prev = i
            year_interval_next = i+1
            break

    #create a list to store the plotted points of the given year
    collection_plot_points=[]
    #after getting the year interval, get the line of the previous year coastline
    for multiline in dict_points["coast_"+str(year_with_data[year_interval_prev])]:

        collection_plot_of_this_multiline=[]

        #after get the line, get the point of the line
        for i,point in enumerate(multiline):
            # no need to draw the vretical line on the first point
            if i==0:
                continue
            #if the point is the last point of the line, no need to draw the vertical line, move to the next line
            elif i==len(multiline)-1:
                break
            else:
                #this segment line may have more than one intersection point with the next year coastline, create a list to store the intersection points
                all_intersection_points=[]
                #get coordination of the two points of the segment line
                start1_lon=multiline[i-1][0]
                start1_lat=multiline[i-1][1]
                start2_lon=multiline[i][0]
                start2_lat=multiline[i][1]
                #iterate the line of the next year, find the line that intersects with the vertical line
                for multiline2 in dict_points["coast_"+str(year_with_data[year_interval_next])]:
                    for j,point2 in enumerate(multiline2):
                        if j==0:
                            continue
                        elif j==len(multiline2)-1:
                            break
                        else:
                            target1_lon=multiline2[j-1][0]
                            target1_lat=multiline2[j-1][1]
                            target2_lon=multiline2[j][0]
                            target2_lat=multiline2[j][1]
                            #if the vertical line does not intersect with the line of the next year, move to the next line
                            if intersect_pt(start1_lon,start1_lat,start2_lon,start2_lat,target1_lon,target1_lat,target2_lon,target2_lat) == "No intersection":
                                continue
                            #if the vertical line intersects with the line of the next year, get the intersection point
                            if intersect_pt(start1_lon,start1_lat,start2_lon,start2_lat,target1_lon,target1_lat,target2_lon,target2_lat) != "No intersection":
                                #get the intersection point of the vertical line
                                intersection_point=intersect_pt(start1_lon,start1_lat,start2_lon,start2_lat,target1_lon,target1_lat,target2_lon,target2_lat)
                                #get the longitude and latitude of the intersection point
                                intersection_lon=intersection_point[0]
                                intersection_lat=intersection_point[1]
                                #determine if the intersction point is on the segment of the line of the next year
                                #if the intersection point is on the segment of the line of the next year, add the intersection point to the list
                                if intersection_lon>=min(target1_lon,target2_lon) and intersection_lon<=max(target1_lon,target2_lon):
                                    all_intersection_points.append(intersection_point)


                #compare the intersection points to get the intersection point that is closest to the current point
                #if there is no intersection point, move to the next line
                if len(all_intersection_points)==0:
                    continue
                #if there are intersection points, select the intersection point that is closest to the current point
                else:
                    selected_distance=0
                    selected_point=all_intersection_points[0]
                    for point in all_intersection_points:
                        distance=math.sqrt((point[0]-start2_lon)**2+(point[1]-start2_lat)**2)
                        if distance>selected_distance:
                            selected_distance=distance
                            selected_point=point        #this is the intersection point

                # get the point that should be plotted
                plotted_point_lon = start2_lon+(selected_point[0]-start2_lon)/(year_with_data[year_interval_next]-year_with_data[year_interval_prev])*(year-year_with_data[year_interval_prev])
                plotted_point_lat = start2_lat+(selected_point[1]-start2_lat)/(year_with_data[year_interval_next]-year_with_data[year_interval_prev])*(year-year_with_data[year_interval_prev])

                plot_point=(plotted_point_lon,plotted_point_lat)
                
                #add the point to the list
                collection_plot_of_this_multiline.append(plot_point)

        #add the list of the line to the list
        collection_plot_points.append(collection_plot_of_this_multiline)


                #get the intersection point of the vertical line from the previous year to the current year
                intersection_point=intersect_pt(start1_lon,start1_lat,start2_lon,start2_lat,target1_lon,target1_lat,target2_lon,target2_lat)
                #if there is an intersection point, plot the line from the intersection point to the current point
                if intersection_point != "No intersection":
                    print(intersection_point)
            

In [49]:
#test cell, can be deleted later
k=3  #slope

x, y= symbols("x y")



eq = [(y-3)/(x-4)-(3-2)/(4-1),(y-4)/(x-3)+1/k]
result = solve(eq, [x, y])
print(result)
return_result=(result[x],result[y])
print(return_result)
print(type(return_result))

{x: 5.00000000000000, y: 3.33333333333333}
(5.00000000000000, 3.33333333333333)
<class 'tuple'>


In [29]:
import geopy.distance

In [36]:
# manipulate the point, give the coordination of the point (pt), the distance, and the direction, return the new coordination.

def point_manipulation(pt,dist,direction):    #pt is a single point with lon,lat; dist is given in meter, direction is given in bearing degree

         
    new_point=geopy.distance.distance(meters=dist).destination((pt[1], pt[0]), bearing=direction)     #(points[1], points[0]), unlike what we see in the point_collection, here the points are given by lat,lon
                                                                                              #Bearing in degrees: 0 – North, 90 – East, 180 – South, 270 or -90 – West.
    return (new_point[1],new_point[0])    
        


In [37]:
#test the function "point_manipulation"
point_manipulation((-124.09567872199995, 46.87866233700004),10,270)

(-124.09580990737436, 46.87866233692487)

In [4]:
coast_1860.explore("Shape_Leng", legend=False)

/usr/local/lib/python3.9/site-packages/mapclassify/classifiers.py:891: RuntimeWarning: invalid value encountered in double_scalars
  gadf = 1 - self.adcm / adam


In [41]:
f_coast_1911.explore()

In [3]:
import json

In [8]:
#/Users/ziyangliu/Desktop/Workspace/Grays_Harbor_Storymap/assets/f_1911_coastline_bay.geojson
f_1911=open("/Users/ziyangliu/Desktop/Workspace/Grays_Harbor_Storymap/assets/f_1911_coastline.geojson")
data_1911=json.load(f_1911)
f_1860=open("/Users/ziyangliu/Desktop/Workspace/Grays_Harbor_Storymap/assets/1860_coastline.geojson")
data_1860=json.load(f_1860)



In [ ]:
entries_1860=data_1860.get("features")
entries_1911=data_1911.get("features")
entries_1860=entries_1860[0].get("geometry").get("coordinates")
print(len(entries_1860[1]))
print(entries_1860[1])
#print(len(entries_1860[0]))

In [ ]:
lines_1911=[]
for i in range(len(entries_1911)):
    each_entry_line_strings=entries_1911[i].get("geometry").get("coordinates")
    for j in range(len(each_entry_line_strings)):
        lines_1911.append(each_entry_line_strings[j])

print(lines_1911)



In [29]:
print(len(lines_1911))

646


In [43]:
lines_1911[0][2][0]

-124.0281589

In [36]:
def findIntersection(line1):
    intersect_pt=[]
    for i in range(len(lines_1911)):
        for j in range(len(lines_1911[i])-1):
            seg=gm.Segment(gm.Point(lines_1911[i][j][0],lines_1911[i][j][1]),gm.Point(lines_1911[i][j+1][0],lines_1911[i][j+1][1]))
            intersect_pt.extend(line1.intersection(seg))
    return intersect_pt



In [40]:
segment=gm.Line(gm.Point(entries_1860[0][0][0],entries_1860[0][0][1]),gm.Point(entries_1860[0][1][0],entries_1860[0][1][1]))
prepend_line=segment.perpendicular_line(gm.Point(entries_1860[0][1][0],entries_1860[0][1][1]))
#print(prepend_line.evalf())
prepend_line.intersection(gm.Segment(gm.Point(lines_1911[0][0][0],lines_1911[0][0][1]),gm.Point(lines_1911[0][1][0],lines_1911[0][1][1])))
#findIntersection(prepend_line)

[]

In [33]:
#test cell
l1=[3]
l2=[9]
l3=[]
l4=[]
l2.extend(l1)
l2.extend(l3)
l3.extend(l4)
print(l2)
print(lines_1911[0][0][0])

[9, 3]
-124.0278177


In [ ]:
for i in range(len(entries_1860)):
    for j in range(len(entries_1860[i])-1):
        segment=gm.Line(gm.Point(entries_1860[i][j][0],entries_1860[i][j][1]),gm.Point(entries_1860[i][j+1][0],entries_1860[i][j+1][1]))
        prepend_line=segment.prependicular_line(gm.Point(entries_1860[i][j+1][0],entries_1860[i][j+1][1]))
        intersections=findIntersection(prepend_line)
        



In [ ]:
for year in range(1861,1911):
    fileName="/Users/ziyangliu/Desktop/Workspace/Grays_Harbor_Storymap/assets/f_"+str(year)+"_coastline.geojson"
    with open(fileName, "w") as outfile:
        json.dump(d,outfile)